In [1]:
import os
import sys
import json
from sklearn.model_selection import train_test_split
from mmengine.config import Config
from mmengine.runner import Runner
from mmengine.hooks import EarlyStoppingHook
from mmengine.visualization import Visualizer, WandbVisBackend
from mmdet.utils import setup_cache_size_limit_of_dynamo
import datetime
import wandb

# mmdetection을 시스템 경로에 추가
sys.path.insert(0, "../mmdetection")

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# 사용자 맞춤형 학습 설정
config_path = './configs/dino/dino-5scale_swin-l_8xb2-12e_coco.py'  # DINO 설정 파일 경로
work_dir = './work_dirs/dino_custom'  # 로그와 모델을 저장할 디렉토리 경로
train_data_root = '/data/ephemeral/home/dataset/'  # 학습 데이터 경로
original_ann_file = '/data/ephemeral/home/dataset/train.json'  # 전체 데이터 어노테이션 파일 경로


In [3]:
# 학습 및 검증 데이터셋 나누기
with open(original_ann_file, 'r') as f:
    annotations = json.load(f)

# 이미지 ID 리스트 가져오기
image_ids = [image['id'] for image in annotations['images']]

# 학습 및 검증 셋으로 분할 (80% 학습, 20% 검증)
train_ids, val_ids = train_test_split(image_ids, test_size=0.2, random_state=42)

# 학습 및 검증 데이터 어노테이션 생성
train_annotations = {
    'images': [img for img in annotations['images'] if img['id'] in train_ids],
    'annotations': [ann for ann in annotations['annotations'] if ann['image_id'] in train_ids],
    'categories': annotations['categories']
}
val_annotations = {
    'images': [img for img in annotations['images'] if img['id'] in val_ids],
    'annotations': [ann for ann in annotations['annotations'] if ann['image_id'] in val_ids],
    'categories': annotations['categories']
}

# 분할된 어노테이션을 파일로 저장
train_ann_file = '/data/ephemeral/home/dataset/train_split.json'
val_ann_file = '/data/ephemeral/home/dataset/val_split.json'

with open(train_ann_file, 'w') as f:
    json.dump(train_annotations, f)
with open(val_ann_file, 'w') as f:
    json.dump(val_annotations, f)

In [4]:
# 어노테이션 파일 수정: category_id를 1부터 시작하도록 조정
def increment_category_id(annotation_file):
    with open(annotation_file, 'r') as f:
        data = json.load(f)

    # 'categories' 섹션의 'id'를 1부터 시작하도록 수정
    id_mapping = {}
    for category in data['categories']:
        old_id = category['id']
        new_id = old_id + 1
        id_mapping[old_id] = new_id
        category['id'] = new_id

    # 'annotations' 섹션의 'category_id'를 매핑에 따라 수정
    for ann in data['annotations']:
        old_cat_id = ann['category_id']
        if old_cat_id in id_mapping:
            ann['category_id'] = id_mapping[old_cat_id]
        else:
            print(f"Warning: annotation id {ann['id']} has invalid category_id {old_cat_id}")

    # 수정된 데이터를 원래 파일에 저장
    with open(annotation_file, 'w') as f:
        json.dump(data, f, indent=4)

# 어노테이션 파일에 category_id를 1부터 시작하도록 수정
increment_category_id(train_ann_file)
increment_category_id(val_ann_file)
increment_category_id(original_ann_file)


In [5]:
# 클래스 설정
classes = (
    "General trash", "Paper", "Paper pack", "Metal", "Glass",
    "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing"
)
num_classes = len(classes)  # 10

In [6]:
# 설정 파일 로드
cfg = Config.fromfile(config_path)

# 작업 디렉토리 수정
cfg.work_dir = work_dir  # 로그와 모델 저장을 위한 작업 디렉토리 수정

# WandB 초기화 (Runner 전에)
run_name = f'dino-5scale_last_layer_{datetime.datetime.now().strftime("%Y%m%d_%H%M%S")}'
wandb.init(
    project='Object_detection',
    name=run_name,
    config=cfg.to_dict(),
    allow_val_change=True,
    reinit=True
)
print(f"WandB Run Initialized: {run_name}")

# WandbVisBackend 설정
wandb_vis_backend = dict(
    type='WandbVisBackend',
    save_dir=cfg.work_dir,  # 저장할 디렉토리
    init_kwargs=dict(
        project='Object_detection',  # WandB 프로젝트 이름
        name=run_name,                # 고유한 실행 이름
        allow_val_change=True         # 설정 값 변경 허용
    ),
    define_metric_cfg=None,
    commit=True,
    log_code_name=None,
    watch_kwargs=None
)

# Visualizer 설정을 Runner의 visualizer 필드로 추가
cfg.visualizer = dict(
    type='Visualizer',
    vis_backends=[
        wandb_vis_backend  # WandbVisBackend 추가
    ],
    name='visualizer'  # Visualizer 이름 (옵션)
)
cfg.log_level="INFO"
cfg.load_from='https://download.openmmlab.com/mmdetection/v3.0/dino/dino-5scale_swin-l_8xb2-12e_coco/dino-5scale_swin-l_8xb2-12e_coco_20230228_072924-a654145f.pth'

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: good117454 (good117454-sejong-university). Use `wandb login --relogin` to force relogin


WandB Run Initialized: dino-5scale_last_layer_20241023_132224


In [7]:
# 데이터셋 루트 경로 수정
#cfg.train_dataloader.dataset.ann_file = train_ann_file  # 학습 데이터 어노테이션 파일 경로 수정
cfg.train_dataloader.dataset.ann_file = original_ann_file
cfg.train_dataloader.dataset.data_prefix = dict(img=train_data_root)  # 학습 데이터 이미지 경로 수정

cfg.val_dataloader.dataset.ann_file = val_ann_file  # 검증 데이터 어노테이션 파일 경로 수정
cfg.val_dataloader.dataset.data_prefix = dict(img=train_data_root)  # 검증 데이터 이미지 경로 수정

# 클래스 설정을 데이터셋의 metainfo에 추가
cfg.train_dataloader.dataset.metainfo = dict(classes=classes)  # 학습 데이터셋 클래스 설정
cfg.val_dataloader.dataset.metainfo = dict(classes=classes)    # 검증 데이터셋 클래스 설정

# 클래스 수 수정
cfg.model.bbox_head.num_classes = num_classes  # 모델의 클래스 수 수정

In [8]:
# 자동 혼합 정밀도 학습 사용 설정
use_amp = True  # 자동 혼합 정밀도(AMP) 학습 사용 여부
if use_amp:
    cfg.optim_wrapper.type = 'AmpOptimWrapper'
    cfg.optim_wrapper.loss_scale = 'dynamic'

# 학습률 자동 스케일링 설정
auto_scale_lr = False  # 학습률 자동 스케일링 사용 여부
if auto_scale_lr:
    if 'auto_scale_lr' in cfg and 'enable' in cfg.auto_scale_lr and 'base_batch_size' in cfg.auto_scale_lr:
        cfg.auto_scale_lr.enable = True
    else:
        raise RuntimeError('설정 파일에 "auto_scale_lr" 또는 필요한 키가 없습니다.')


In [9]:
# 데이터 로더 설정 최적화
cfg.train_dataloader.batch_size = 1  # 배치 사이즈를 1로 줄임
cfg.val_dataloader.batch_size = 1

cfg.train_dataloader.num_workers = 2  # 워커 수 줄이기
cfg.val_dataloader.num_workers = 2

# Prefetch factor와 persistent_workers 설정
cfg.train_dataloader.prefetch_factor = 2
cfg.train_dataloader.persistent_workers = False
cfg.val_dataloader.prefetch_factor = 2
cfg.val_dataloader.persistent_workers = False

In [10]:
# 학습 재개 설정
cfg.param_scheduler = [
    dict(
        type='LinearLR', start_factor=0.001, by_epoch=False, begin=0, end=500),
    dict(
        type='MultiStepLR',
        begin=0,
        end=14,
        by_epoch=True,
        milestones=[11],
        gamma=0.1)
]
cfg.max_epochs=14
cfg.bboxhead.classes=10
resume_training = False  # 학습 재개 여부 설정 (True로 설정 시 학습 재개)
resume_checkpoint_path = None  # 학습 재개 시 체크포인트 경로 지정
if resume_training:
    if resume_checkpoint_path is None:
        cfg.resume = True
        cfg.load_from = None
    else:
        cfg.resume = True
        cfg.load_from = resume_checkpoint_path

In [11]:
# 반복적인 컴파일 횟수를 줄여 학습 속도 향상
setup_cache_size_limit_of_dynamo()

In [12]:
# 평가 설정 추가 및 수정
cfg.evaluation = dict(interval=1, metric='bbox', save_best='auto')
# 평가자(evaluator) 설정 수정
if hasattr(cfg, 'val_evaluator'):
    # 평가자가 딕셔너리 또는 리스트로 정의되어 있을 경우
    if isinstance(cfg.val_evaluator, dict):
        cfg.val_evaluator.ann_file = val_ann_file
    elif isinstance(cfg.val_evaluator, list):
        for evaluator in cfg.val_evaluator:
            evaluator['ann_file'] = val_ann_file
elif 'evaluation' in cfg:
    # 평가 섹션 내에 ann_file을 추가
    cfg.evaluation['ann_file'] = val_ann_file


In [13]:
# 설정 파일을 통해 러너 생성
runner = Runner.from_cfg(cfg)

# 학습 시작 전에 검증 어노테이션 파일 존재 여부 확인
if not os.path.exists(val_ann_file):
    raise FileNotFoundError(f'Validation annotation file not found: {val_ann_file}')

# 학습 시작
runner.train()

10/23 13:22:52 - mmengine - INFO - 
------------------------------------------------------------
System environment:
    sys.platform: linux
    Python: 3.10.13 (main, Sep 11 2023, 13:44:35) [GCC 11.2.0]
    CUDA available: True
    MUSA available: False
    numpy_random_seed: 901828237
    GPU 0: Tesla V100-SXM2-32GB
    CUDA_HOME: None
    GCC: gcc (Ubuntu 9.4.0-1ubuntu1~20.04.2) 9.4.0
    PyTorch: 1.12.1+cu116
    PyTorch compiling details: PyTorch built with:
  - GCC 9.3
  - C++ Version: 201402
  - Intel(R) Math Kernel Library Version 2020.0.0 Product Build 20191122 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v2.6.0 (Git Hash 52b5f107dd9cf10910aaa19cb47f3abf9b349815)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX2
  - CUDA Runtime 11.6
  - NVCC architecture flags: -gencode;arch=compute_37,code=sm_37;-gencode;arch=compute_50,code=sm_50;-gencode;arch=compute_60,code=sm_60;-ge

10/23 13:23:10 - mmengine - INFO - Distributed training is not used, all SyncBatchNorm (SyncBN) layers in the model will be automatically reverted to BatchNormXd layers if they are used.
10/23 13:23:10 - mmengine - INFO - Hooks will be executed in the following order:
before_run:
(VERY_HIGH   ) RuntimeInfoHook                    
(BELOW_NORMAL) LoggerHook                         
 -------------------- 
before_train:
(VERY_HIGH   ) RuntimeInfoHook                    
(NORMAL      ) IterTimerHook                      
(VERY_LOW    ) CheckpointHook                     
 -------------------- 
before_train_epoch:
(VERY_HIGH   ) RuntimeInfoHook                    
(NORMAL      ) IterTimerHook                      
(NORMAL      ) DistSamplerSeedHook                
 -------------------- 
before_train_iter:
(VERY_HIGH   ) RuntimeInfoHook                    
(NORMAL      ) IterTimerHook                      
 -------------------- 
after_train_iter:
(VERY_HIGH   ) RuntimeInfoHook                

/data/ephemeral/home/mmdetection/mmdet/models/layers/positional_encoding.py:103: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  dim_t = self.temperature**(2 * (dim_t // 2) / self.num_feats)
/opt/conda/lib/python3.10/site-packages/torch/functional.py:478: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2894.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
/data/ephemeral/home/mmdetection/mmdet/models/layers/transformer/utils.py:71: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future versio

10/23 13:23:52 - mmengine - INFO - Epoch(train)  [1][  50/3906]  base_lr: 9.9098e-06 lr: 9.9098e-06  eta: 10:33:01  time: 0.6952  data_time: 0.0118  memory: 9376  grad_norm: nan  loss: 23.0354  loss_cls: 1.0717  loss_bbox: 0.2223  loss_iou: 0.2524  d0.loss_cls: 1.0871  d0.loss_bbox: 0.3106  d0.loss_iou: 0.3513  d1.loss_cls: 1.0456  d1.loss_bbox: 0.2788  d1.loss_iou: 0.2970  d2.loss_cls: 1.0494  d2.loss_bbox: 0.2487  d2.loss_iou: 0.2879  d3.loss_cls: 1.0817  d3.loss_bbox: 0.2393  d3.loss_iou: 0.2667  d4.loss_cls: 1.0382  d4.loss_bbox: 0.2223  d4.loss_iou: 0.2527  enc_loss_cls: 1.0525  enc_loss_bbox: 0.4354  enc_loss_iou: 0.4511  dn_loss_cls: 1.0441  dn_loss_bbox: 0.4100  dn_loss_iou: 0.3051  d0.dn_loss_cls: 1.0956  d0.dn_loss_bbox: 0.8001  d0.dn_loss_iou: 0.5500  d1.dn_loss_cls: 1.0716  d1.dn_loss_bbox: 0.4987  d1.dn_loss_iou: 0.3699  d2.dn_loss_cls: 1.0492  d2.dn_loss_bbox: 0.4325  d2.dn_loss_iou: 0.3247  d3.dn_loss_cls: 1.0659  d3.dn_loss_bbox: 0.4107  d3.dn_loss_iou: 0.3095  d4.dn_lo

/data/ephemeral/home/mmdetection/mmdet/models/layers/positional_encoding.py:103: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  dim_t = self.temperature**(2 * (dim_t // 2) / self.num_feats)
/data/ephemeral/home/mmdetection/mmdet/models/layers/transformer/utils.py:71: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  dim_t = temperature**(2

...

10/23 23:32:26 - mmengine - INFO - Epoch(val) [13][ 50/977]    eta: 0:03:56  time: 0.2552  data_time: 0.0074  memory: 8711  
10/23 23:32:38 - mmengine - INFO - Epoch(val) [13][100/977]    eta: 0:03:42  time: 0.2517  data_time: 0.0026  memory: 4128  
10/23 23:32:51 - mmengine - INFO - Epoch(val) [13][150/977]    eta: 0:03:29  time: 0.2516  data_time: 0.0025  memory: 4128  
10/23 23:33:03 - mmengine - INFO - Epoch(val) [13][200/977]    eta: 0:03:16  time: 0.2522  data_time: 0.0024  memory: 4128  
10/23 23:33:16 - mmengine - INFO - Epoch(val) [13][250/977]    eta: 0:03:03  time: 0.2540  data_time: 0.0027  memory: 4128  
10/23 23:33:29 - mmengine - INFO - Epoch(val) [13][300/977]    eta: 0:02:51  time: 0.2579  data_time: 0.0027  memory: 4128  
10/23 23:33:42 - mmengine - INFO - Epoch(val) [13][350/977]    eta: 0:02:39  time: 0.2567  data_time: 0.0027  memory: 4128  
10/23 23:33:54 - mmengine - INFO - Epoch(val) [13][400/977]    eta: 0:02:26  time: 0.2512  data_time: 0.0025  memory: 4128  


/data/ephemeral/home/mmdetection/mmdet/models/layers/positional_encoding.py:103: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  dim_t = self.temperature**(2 * (dim_t // 2) / self.num_feats)
/data/ephemeral/home/mmdetection/mmdet/models/layers/transformer/utils.py:71: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  dim_t = temperature**(2

10/24 00:19:58 - mmengine - INFO - Epoch(val) [14][ 50/977]    eta: 0:03:58  time: 0.2571  data_time: 0.0058  memory: 8711  
10/24 00:20:10 - mmengine - INFO - Epoch(val) [14][100/977]    eta: 0:03:43  time: 0.2531  data_time: 0.0029  memory: 4128  
10/24 00:20:23 - mmengine - INFO - Epoch(val) [14][150/977]    eta: 0:03:29  time: 0.2507  data_time: 0.0024  memory: 4128  
10/24 00:20:35 - mmengine - INFO - Epoch(val) [14][200/977]    eta: 0:03:16  time: 0.2514  data_time: 0.0025  memory: 4128  
10/24 00:20:48 - mmengine - INFO - Epoch(val) [14][250/977]    eta: 0:03:03  time: 0.2520  data_time: 0.0025  memory: 4128  
10/24 00:21:00 - mmengine - INFO - Epoch(val) [14][300/977]    eta: 0:02:50  time: 0.2504  data_time: 0.0024  memory: 4128  
10/24 00:21:13 - mmengine - INFO - Epoch(val) [14][350/977]    eta: 0:02:38  time: 0.2514  data_time: 0.0024  memory: 4128  
10/24 00:21:26 - mmengine - INFO - Epoch(val) [14][400/977]    eta: 0:02:25  time: 0.2524  data_time: 0.0025  memory: 4128  


base_lr,█████████████████████████████████▁▁▁▁▁▁▁
coco/bbox_mAP,▁▄▅▅▅▆▆▆▅▅▅▇██
coco/bbox_mAP_50,▁▄▅▆▆▆▆▆▅▅▆▇██
coco/bbox_mAP_75,▁▄▅▅▅▆▆▆▅▅▅▇██
coco/bbox_mAP_l,▁▄▅▆▅▆▆▆▆▅▅▇██
coco/bbox_mAP_m,▁▂▃▄▃▆▅▅▃▄▆▇██
coco/bbox_mAP_s,▁▆▅▇█▂▄▄▄█▂▅█▇
d0.dn_loss_bbox,█▇█▇▇█▇▆▅▅▅▆▄▅▅▅▄▅▄▄▄▆▃▄▄▅▄▅▄▃▂▃▃▃▂▃▁▂▂▃
d0.dn_loss_cls,█▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
d0.dn_loss_iou,▆▇▇▆▅▇██▅▅▅▃▅▆▅▅▄▅▄▃▅▅▅▃▅▅▅▃▄▄▁▂▂▂▃▃▃▃▂▁
d0.loss_bbox,▆▅█▆▅▅▅▅▃▅▄▃▄▄▄▃▃▃▄▃▅▄▅▄▂▄▄▃▃▂▂▃▁▂▁▂▁▂▁▂


DINO(
  (data_preprocessor): DetDataPreprocessor()
  (backbone): SwinTransformer(
    (patch_embed): PatchEmbed(
      (adap_padding): AdaptivePadding()
      (projection): Conv2d(3, 192, kernel_size=(4, 4), stride=(4, 4))
      (norm): LayerNorm((192,), eps=1e-05, elementwise_affine=True)
    )
    (drop_after_pos): Dropout(p=0.0, inplace=False)
    (stages): ModuleList(
      (0): SwinBlockSequence(
        (blocks): ModuleList(
          (0): SwinBlock(
            (norm1): LayerNorm((192,), eps=1e-05, elementwise_affine=True)
            (attn): ShiftWindowMSA(
              (w_msa): WindowMSA(
                (qkv): Linear(in_features=192, out_features=576, bias=True)
                (attn_drop): Dropout(p=0.0, inplace=False)
                (proj): Linear(in_features=192, out_features=192, bias=True)
                (proj_drop): Dropout(p=0.0, inplace=False)
                (softmax): Softmax(dim=-1)
              )
              (drop): DropPath()
            )
            (norm